In [1]:
from baseball_scraper import baseball_reference
import pandas as pd
import time
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
s = baseball_reference.TeamScraper()
s.set_season(2023)

In [3]:
teams = [
    "ARI", "ATL", "BAL", "BOS", "CHW", "CHC", "CIN", "CLE", "COL", "DET",
    "HOU", "KCR", "LAA", "LAD", "MIA", "MIL", "MIN", "NYY", "NYM", "OAK",
    "PHI", "PIT", "SDP", "SFG", "SEA", "STL", "TBR", "TEX", "TOR", "WSN"
]


all_games = []
for team in teams:
    df = s.scrape(team)
    time.sleep(5)
    all_games.append(df)

df = pd.concat(all_games)
df = df[df['W/L'].notna()]
division_mapping = {
    "BAL": "AL East", "TOR": "AL East", "NYY": "AL East", "BOS": "AL East", "TBR": "AL East",
    "CHW": "AL Central", "CLE": "AL Central", "DET": "AL Central", "KCR": "AL Central", "MIN": "AL Central",
    "HOU": "AL West", "TEX": "AL West", "LAA": "AL West", "OAK": "AL West", "SEA": "AL West",
    "NYM": "NL East", "WSN": "NL East", "PHI": "NL East", "MIA": "NL East", "ATL": "NL East",
    "CHC": "NL Central", "CIN": "NL Central", "STL": "NL Central", "PIT": "NL Central", "MIL": "NL Central",
    "ARI": "NL West", "LAD": "NL West", "COL": "NL West", "SFG": "NL West", "SDP": "NL West"
}

# Create the 'Tm_div' column using the mapping dictionary
df['Tm_div'] = df['Tm'].map(division_mapping)
df['Opp_div'] = df['Opp'].map(division_mapping)
# Assuming df is your dataframe
df['Run_Differential'] = df['R'] - df['RA']
# Ensure 'Date' is in datetime format
df['Date'] = pd.to_datetime(df['Date'])

# Extract the year
df['Year'] = df['Date'].dt.year

/Users/curtisschaefer/opt/anaconda3/lib/python3.9/site-packages/baseball_scraper/baseball_reference.py:136: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  data = data.drop('', 1)  # not a useful column
/Users/curtisschaefer/opt/anaconda3/lib/python3.9/site-packages/baseball_scraper/baseball_reference.py:221: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  data = data.drop('Streak2', 1)
/Users/curtisschaefer/opt/anaconda3/lib/python3.9/site-packages/baseball_scraper/baseball_reference.py:136: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  data = data.drop('', 1)  # not a useful column
/Users/curtisschaefer/opt/anaconda3/lib/python3.9/site-packages/baseball_scraper/baseball_reference.py:221: FutureWarning: In a future version of pan

/Users/curtisschaefer/opt/anaconda3/lib/python3.9/site-packages/baseball_scraper/baseball_reference.py:136: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  data = data.drop('', 1)  # not a useful column
/Users/curtisschaefer/opt/anaconda3/lib/python3.9/site-packages/baseball_scraper/baseball_reference.py:221: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  data = data.drop('Streak2', 1)
/Users/curtisschaefer/opt/anaconda3/lib/python3.9/site-packages/baseball_scraper/baseball_reference.py:136: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  data = data.drop('', 1)  # not a useful column
/Users/curtisschaefer/opt/anaconda3/lib/python3.9/site-packages/baseball_scraper/baseball_reference.py:221: FutureWarning: In a future version of pan

In [4]:
# Group by division and calculate run differential
division_run_differential = df.groupby('Tm_div')['Run_Differential'].sum().reset_index()

# Convert DataFrame to dictionary
division_run_differential_dict = division_run_differential.set_index('Tm_div')['Run_Differential'].to_dict()

# Print the dictionary
print(division_run_differential_dict)

{'AL Central': -334.0, 'AL East': 311.0, 'AL West': 0.0, 'NL Central': -96.0, 'NL East': 92.0, 'NL West': 27.0}


In [5]:
games = df[['Tm_div', 'Opp_div', 'Run_Differential']]

# Define the desired order
division_order = ['AL East', 'AL Central', 'AL West', 'NL East', 'NL Central', 'NL West']

# Group by 'Tm_div' and 'Opp_div' and sum the run differentials
division_vs_division = games.groupby(['Tm_div', 'Opp_div'])['Run_Differential'].sum().reset_index()

# Pivot the table to get a matrix of run differentials
run_diff_matrix = division_vs_division.pivot(index='Tm_div', columns='Opp_div', values='Run_Differential')

# Fill NaN values with zeros and convert to integer
run_diff_matrix = run_diff_matrix.fillna(0).astype(int)
run_diff_matrix = run_diff_matrix.reindex(index=division_order, columns=division_order)

# Calculate 'Total' column by summing across rows, excluding self vs self match (diagonal)
np.fill_diagonal(run_diff_matrix.values, 0)
run_diff_matrix['Total'] = run_diff_matrix.sum(axis=1)

# Convert 'Total' column to int
run_diff_matrix['Total'] = run_diff_matrix['Total'].astype(int)
# Rename columns
run_diff_matrix.rename_axis('', axis=0, inplace=True)
run_diff_matrix.rename_axis("", axis=1, inplace=True)


run_diff_matrix


,AL East,AL Central,AL West,NL East,NL Central,NL West,Total
,,,,,,,
AL East,0,165,73,4,37,32,311
AL Central,-165,0,-89,-56,19,-43,-334
AL West,-73,89,0,-33,18,-1,0
NL East,-4,56,33,0,5,2,92
NL Central,-37,-19,-18,-5,0,-17,-96
NL West,-32,43,1,-2,17,0,27
